In [ ]:
!pip install transformers datasets pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoTokenizer, RobertaForQuestionAnswering
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
# check device
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

print(f"Using {device} device")

Using cuda device


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'CSE-156/Analysis Study'

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/CSE-156/Analysis Study


In [ ]:
squad_train = load_dataset("rajpurkar/squad", split="train")
squad_validation = load_dataset("rajpurkar/squad", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
squad_train[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [ ]:
# convert dataset to pd dataframe
squad_df = squad_validation.to_pandas()
squad_df["prediction"] = "NO-OUTPUT"
squad_df

,id,title,context,question,answers,prediction
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",NO-OUTPUT
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"{'text': ['Carolina Panthers', 'Carolina Panth...",NO-OUTPUT
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"{'text': ['Santa Clara, California', 'Levi's S...",NO-OUTPUT
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",NO-OUTPUT
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",NO-OUTPUT
...,...,...,...,...,...,...
10565,5737aafd1c456719005744fb,Force,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,"{'text': ['kilogram-force', 'pound-force', 'ki...",NO-OUTPUT
10566,5737aafd1c456719005744fc,Force,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,"{'text': ['kilopond', 'kilopond', 'kilopond', ...",NO-OUTPUT
10567,5737aafd1c456719005744fd,Force,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,"{'text': ['slug', 'metric slug', 'metric slug'...",NO-OUTPUT
10568,5737aafd1c456719005744fe,Force,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,"{'text': ['kip', 'kip', 'kip', 'kip', 'kip'], ...",NO-OUTPUT


In [ ]:
# shuffle the dataset
squad_df = squad_df.sample(frac=1).reset_index(drop=True)
squad_df.head()

,id,title,context,question,answers,prediction
0,57105da9a58dae1900cd69a1,Huguenot,Huguenot numbers peaked near an estimated two ...,What treaty ended the Wars of Religion?,"{'text': ['the Edict of Nantes', 'Edict of Nan...",NO-OUTPUT
1,57094b4f9928a814004714f6,Sky_(United_Kingdom),While BSkyB had been excluded from being a par...,What consortium was BSkyB excluded from?,"{'text': ['ONdigital', 'ONdigital', 'ONdigital...",NO-OUTPUT
2,572905ce1d04691400778f84,Immune_system,Phagocytosis is an important feature of cellul...,Phagocytes can be called to a specific locatio...,"{'text': ['cytokines', 'cytokines', 'cytokines...",NO-OUTPUT
3,57282358ff5b5019007d9dde,"Jacksonville,_Florida",Mean monthly temperatures range from around 53...,What is Jacksonville's hottest recorded temper...,"{'text': ['104 °F', '104 °F', '104 °F'], 'answ...",NO-OUTPUT
4,571caac55efbb31900334dc9,Oxygen,By the late 19th century scientists realized t...,What minor amount of liquid oxygen was produce...,"{'text': ['few drops', 'a few drops', 'a few d...",NO-OUTPUT


In [ ]:
squad_df.shape[0]

10570

In [ ]:
# initialize model and tokenizer for roberta model for QA
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
# method to call model, get prediction and return output
def get_answer(data: pd.Series):
  question, context = data["question"], data["context"]
  inputs = tokenizer(question, context, return_tensors="pt", truncation=True).to(device)
  outputs = model(**inputs)
  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
  return answer


In [ ]:
data = squad_df.iloc[141]
id = data["id"]
answer = data["answers"]["text"][0]
squad_df.at[141, "prediction"] = "test"
print(f"data point:\n{squad_df.iloc[141]}\nanswer: {answer}")

data point:
id                                     570d2556fed7b91900d45c4b
title                                      Victoria_(Australia)
context       Immigrants arrived from all over the world to ...
question      What were the conditions for miners in the gol...
answers       {'text': ['cramped and unsanitary', 'cramped a...
prediction                                                 test
Name: 141, dtype: object
answer: cramped and unsanitary


In [ ]:
# run model on entire squad dataset -> put prediction into new column
for index in tqdm(range(squad_df.shape[0])):
  data = squad_df.iloc[index]
  answer = data["answers"]["text"][0]
  squad_df.at[index, "prediction"] = get_answer(data)
squad_df.head()

100%|██████████| 10570/10570 [03:21<00:00, 52.40it/s]


,id,title,context,question,answers,prediction
0,57105da9a58dae1900cd69a1,Huguenot,Huguenot numbers peaked near an estimated two ...,What treaty ended the Wars of Religion?,"{'text': ['the Edict of Nantes', 'Edict of Nan...",Edict of Nantes
1,57094b4f9928a814004714f6,Sky_(United_Kingdom),While BSkyB had been excluded from being a par...,What consortium was BSkyB excluded from?,"{'text': ['ONdigital', 'ONdigital', 'ONdigital...",ONdigital
2,572905ce1d04691400778f84,Immune_system,Phagocytosis is an important feature of cellul...,Phagocytes can be called to a specific locatio...,"{'text': ['cytokines', 'cytokines', 'cytokines...",cytokines
3,57282358ff5b5019007d9dde,"Jacksonville,_Florida",Mean monthly temperatures range from around 53...,What is Jacksonville's hottest recorded temper...,"{'text': ['104 °F', '104 °F', '104 °F'], 'answ...",104 °
4,571caac55efbb31900334dc9,Oxygen,By the late 19th century scientists realized t...,What minor amount of liquid oxygen was produce...,"{'text': ['few drops', 'a few drops', 'a few d...",a few drops


In [ ]:
# checking for blank cases
for index in range(100):
  if squad_df.iloc[index]["prediction"] == "":
    print(f"\nno predction at index: {index}, data point:\n{squad_df.iloc[index]}\n----------")


no predction at index: 50, data point:
id                                     56d6017d1c85041400946ec1
title                                             Super_Bowl_50
context       The Panthers finished the regular season with ...
question      Who did Denver beat in the 2015 AFC Championsh...
answers       {'text': ['New England Patriots', 'the New Eng...
prediction                                                     
Name: 50, dtype: object
----------

no predction at index: 73, data point:
id                                     56d601e41c85041400946ed0
title                                             Super_Bowl_50
context       The Broncos took an early lead in Super Bowl 5...
question      Which Newton turnover resulted in seven points...
answers       {'text': ['a fumble', 'a fumble', 'fumble'], '...
prediction                                                     
Name: 73, dtype: object
----------

no predction at index: 95, data point:
id                                     56d

In [ ]:
# testing index 50
data = squad_df.iloc[50]
context, question, answer = data["context"], data["question"], data["answers"]["text"]
print(f"context: {context}")
print(f"question: {question}")
print(f"answer: {answer}")
get_answer(data)

context: The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl.
question: Who did Denver beat in the 2015 AFC Championship game?
answer: ['New England Patriots' 'the New England Patriots' 'New England Patriots']


''

In [ ]:
# convert squad df to csv file -> attempt to read on diff notebook
squad_df.to_csv('squad_pred.csv', encoding="utf-8", index=False)

In [ ]:
# load squad v2, convert to pandas df, add prediction col, shuffle
squad_v2_validation = load_dataset("rajpurkar/squad_v2", split="validation")
squad_v2_df = squad_v2_validation.to_pandas()
squad_v2_df["prediction"] = "NO-OUTPUT"
squad_v2_df = squad_v2_df.sample(frac=1).reset_index(drop=True)
squad_v2_df

,id,title,context,question,answers,prediction
0,570d2417fed7b91900d45c3e,Victoria_(Australia),The economy of Victoria is highly diversified:...,Where according to gross state product does Vi...,"{'text': ['second', 'second', 'second'], 'answ...",NO-OUTPUT
1,572864542ca10214002da2e1,University_of_Chicago,The Maroons compete in the NCAA's Division III...,What player first won the Heisman Trophy for t...,"{'text': ['Jay Berwanger', 'Jay Berwanger', 'J...",NO-OUTPUT
2,5ad15289645df0001a2d175d,European_Union_law,Free movement of goods within the European Uni...,What were states not responsible for in Articl...,"{'text': [], 'answer_start': []}",NO-OUTPUT
3,5a668fc9f038b7001ab0bfe6,Packet_switching,"Merit Network, Inc., an independent non-profit...",After the University of Ann Arbor and Wayne St...,"{'text': [], 'answer_start': []}",NO-OUTPUT
4,5730035e04bcaa1900d77004,Islamism,Roughly contemporaneous with Maududi was the f...,The motto of the Muslim Brotherhood specifies ...,"{'text': ['the Qur'an', 'the Qur'an', 'Qur'an'...",NO-OUTPUT
...,...,...,...,...,...,...
11868,57264cfa708984140094c1c5,Ctenophora,The outer surface bears usually eight comb row...,What does the 9 +3 pattern of cilia thought to...,"{'text': ['supporting function', 'suspected to...",NO-OUTPUT
11869,5a82370631013a001a3352e2,Harvard_University,"Harvard operates several arts, cultural, and s...",How many museums does the Busch-Reisinger Muse...,"{'text': [], 'answer_start': []}",NO-OUTPUT
11870,5ad4a75aba00c4001a268eea,Immune_system,Microorganisms or toxins that successfully ent...,What kind of cells fail to send out alarm sign...,"{'text': [], 'answer_start': []}",NO-OUTPUT
11871,57378b141c4567190057449f,Force,"Since then, and so far, general relativity has...",What theory best explains gravity?,"{'text': ['general relativity', 'general relat...",NO-OUTPUT


In [ ]:
# example data point from squad v2
squad_v2_df.iloc[0]

id                                     570d2417fed7b91900d45c3e
title                                      Victoria_(Australia)
context       The economy of Victoria is highly diversified:...
question      Where according to gross state product does Vi...
answers       {'text': ['second', 'second', 'second'], 'answ...
prediction                                            NO-OUTPUT
Name: 0, dtype: object

In [ ]:
# run model on entire squad v2 dataset -> put prediction into new column
for index in tqdm(range(squad_v2_df.shape[0])):
  data = squad_v2_df.iloc[index]
  squad_v2_df.at[index, "prediction"] = get_answer(data)
squad_v2_df

100%|██████████| 11873/11873 [03:20<00:00, 59.08it/s]


,id,title,context,question,answers,prediction
0,570d2417fed7b91900d45c3e,Victoria_(Australia),The economy of Victoria is highly diversified:...,Where according to gross state product does Vi...,"{'text': ['second', 'second', 'second'], 'answ...",second
1,572864542ca10214002da2e1,University_of_Chicago,The Maroons compete in the NCAA's Division III...,What player first won the Heisman Trophy for t...,"{'text': ['Jay Berwanger', 'Jay Berwanger', 'J...",Jay Berwanger
2,5ad15289645df0001a2d175d,European_Union_law,Free movement of goods within the European Uni...,What were states not responsible for in Articl...,"{'text': [], 'answer_start': []}",
3,5a668fc9f038b7001ab0bfe6,Packet_switching,"Merit Network, Inc., an independent non-profit...",After the University of Ann Arbor and Wayne St...,"{'text': [], 'answer_start': []}",
4,5730035e04bcaa1900d77004,Islamism,Roughly contemporaneous with Maududi was the f...,The motto of the Muslim Brotherhood specifies ...,"{'text': ['the Qur'an', 'the Qur'an', 'Qur'an'...",the Qur'an
...,...,...,...,...,...,...
11868,57264cfa708984140094c1c5,Ctenophora,The outer surface bears usually eight comb row...,What does the 9 +3 pattern of cilia thought to...,"{'text': ['supporting function', 'suspected to...",What does the 9 +3 pattern of cilia thought to...
11869,5a82370631013a001a3352e2,Harvard_University,"Harvard operates several arts, cultural, and s...",How many museums does the Busch-Reisinger Muse...,"{'text': [], 'answer_start': []}",
11870,5ad4a75aba00c4001a268eea,Immune_system,Microorganisms or toxins that successfully ent...,What kind of cells fail to send out alarm sign...,"{'text': [], 'answer_start': []}",
11871,57378b141c4567190057449f,Force,"Since then, and so far, general relativity has...",What theory best explains gravity?,"{'text': ['general relativity', 'general relat...",general relativity


In [ ]:
# convert squad v2 df to csv file -> attempt to read on diff notebook
squad_v2_df.to_csv('squad_v2_pred.csv', encoding="utf-8", index=False)